In [2]:
import numpy as np

data = np.load("data/toy_data/mutseqs.npy")
embeddings = np.load("data/toy_data/ESMEmbed.npy")


In [3]:
embeddings = np.squeeze(embeddings,axis=1)

In [4]:
print(embeddings.shape)

(188, 163, 1280)


In [5]:
# print(data[:5],data.shape)
# print(embeddings[:5],embeddings.shape)
ngenes = 10
glen = embeddings.shape[1]//ngenes

cutpoints = [glen*i for i in range(1,ngenes)]


cuts = np.split(embeddings,cutpoints,axis=1)

def average_embeddings(slice_):
    return np.mean(slice_,axis=1)


# 

cuts = np.array(list(map(average_embeddings,cuts)))

# x = [[np.mean(cut[0],axis=0),np.mean(cut[1],axis=0)] for cut in cuts]
print(cuts.shape)
np.save("data/toy_data/EmbeddingsDivbyGenes_6.npy",cuts)
# x = [np.concatenate((cut[0],cut[1])) for cut in x]
# x = np.array(x)


(10, 188, 1280)


# Graph Attention

In [6]:
# !pip install networkx
import networkx as nx

graph = nx.erdos_renyi_graph(ngenes, 0.5, seed=2, directed=False)

In [7]:
print(graph.nodes,graph.edges)
# nx.draw(graph)c

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] [(0, 3), (0, 4), (0, 8), (1, 4), (1, 5), (1, 6), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 4), (3, 8), (3, 9), (4, 5), (4, 6), (5, 6), (7, 8)]


In [8]:
import torch
import dgl
from torch import cuda

if cuda.is_available():
    device = torch.device('cuda')
else:
    print("this is running in cpu!")
    device = torch.device('cpu')
from torch_geometric.utils.convert import from_networkx
data = cuts.transpose(1,0,2)
print(data.shape)




/home/ssawmya/anaconda3/envs/gpu_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(188, 10, 1280)


In [136]:
import random
for gene in cuts:
    for patient in gene:
        random_indices = random.sample(range(1, len(patient)), 100)
        max_,min_ = max(patient),min(patient)
        old = patient[random_indices]
        patient[random_indices] = np.random.uniform(min_,max_,size=100)
        new = patient[random_indices]
        assert np.abs(old - new).sum() != 0
        

# max(cuts[0][0]),min(cuts[0][0]))

In [139]:

np.corrcoef(cuts[0])
data = cuts.transpose(1,0,2)
print(data.shape)



(188, 10, 1280)


In [9]:
def build_graph(node_embedding):
    G = from_networkx(graph)
    
    G.x = torch.Tensor(node_embedding)

    G = G.to(device)
    return G
graphs = [build_graph(embedding) for embedding in data]
print(len(graphs))

188


In [10]:
hparams = {
    "input_size":cuts.shape[2],
    "hsz":500,
    "alpha": 0.2,
    "num_heads":5,
    "bsz":16,
    "num_epochs":10,
    "lr":10,
    "ngenes":10
}

In [12]:
phenotype_table = np.load("seq_table.npy",allow_pickle=True)
Y = phenotype_table[:,2].astype(np.int64)
print(Y)
# Y = torch.nn.functional.one_hot(torch.Tensor(Y),num_classes=2)

dataset = list(zip(graphs,Y))
from dgl.data.utils import split_dataset
train_dataset, valid_dataset, test_dataset = split_dataset(dataset, [0.8, 0.1, 0.1])

# from torch.utils.data import DataLoader
# def collate(samples):
#         graphs, labels = map(list, zip(*samples))
#         batched_graph = dgl.batch(graphs)
#         return batched_graph, torch.tensor(labels)
    
# train_loader = DataLoader(train_dataset, batch_size=hparams["bsz"], shuffle=True)
# valid_loader = DataLoader(valid_dataset, hparams["bsz"], shuffle=True,
#                      collate_fn=collate)
# test_loader = DataLoader(test_dataset, hparams["bsz"], shuffle=True,
#                      collate_fn=collate)


[0 0 0 0 0 1 1 0 1 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0 0 1 0 1 0 1 0 1 1 0 1 0 0
 0 0 0 1 0 0 0 1 1 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 1
 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 1 0 1 0 0 0 1 0 0 0 1 0 0 1 1 0 0 1 1 0 1
 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1
 1 0 0 0 0 1 0 1 0 0 1 1 0 1 1 1 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 1 0 0 0 1
 0 0]


In [23]:
from torch_geometric.nn import GCNConv, GATv2Conv
from torch import nn


class GAT_Protein(nn.Module):
    def __init__(self,hparams):
        super(GAT_Protein, self).__init__()
        self.input_layer = nn.Linear(hparams["input_size"],hparams["hsz"])
        self.GATLayer1 = GATv2Conv(hparams["hsz"],hparams["hsz"],num_heads=hparams["num_heads"])
        self.output_layer1 = nn.Linear(hparams["hsz"]*hparams["ngenes"],hparams["hsz"])
        self.output_layer = nn.Linear(hparams["hsz"],1)
        self.leakyrelu = nn.LeakyReLU(hparams["alpha"])
        # self.output_layer = nn.functional.softmax(hsz,1)
        
    def forward(self,graph):
        
        # print(x)
        x = self.input_layer(graph.x)
        
        x = self.leakyrelu(self.GATLayer1(x,graph.edge_index))
        # print("Before Flatten",x,x.shape)
        x = torch.flatten(x)
        # print("After Flatten",x.shape)
        x = self.output_layer1(x)
        x = self.output_layer(x)
        # print("Before Softmax",x,x.shape)
        # x = nn.functional.softmax(x)
        x = nn.functional.softmax(x)
        # print("After Softmax",x)
        return x
    
    def avg_attention_heads(self, x):
        return torch.mean(x, dim = 1)
        
    

In [24]:
from torchsummary import summary
print(hparams)


model = GAT_Protein(hparams).to(device)
summary(model)


{'input_size': 1280, 'hsz': 500, 'alpha': 0.2, 'num_heads': 5, 'bsz': 16, 'num_epochs': 10, 'lr': 10, 'ngenes': 10}
Layer (type:depth-idx)                   Param #
├─Linear: 1-1                            640,500
├─GATv2Conv: 1-2                         --
|    └─SumAggregation: 2-1               --
|    └─Linear: 2-2                       250,500
|    └─Linear: 2-3                       250,500
├─Linear: 1-3                            2,500,500
├─Linear: 1-4                            501
├─LeakyReLU: 1-5                         --
Total params: 3,642,501
Trainable params: 3,642,501
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Linear: 1-1                            640,500
├─GATv2Conv: 1-2                         --
|    └─SumAggregation: 2-1               --
|    └─Linear: 2-2                       250,500
|    └─Linear: 2-3                       250,500
├─Linear: 1-3                            2,500,500
├─Linear: 1-4                            501
├─LeakyReLU: 1-5                         --
Total params: 3,642,501
Trainable params: 3,642,501
Non-trainable params: 0

In [27]:
from tqdm import tqdm
def train(model, train_loader):
    model.train()
    loss_fn = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=hparams["lr"])

    for epoch in range(hparams["num_epochs"]):
        train_loss = []
        
        for iter, (bg, label) in enumerate(tqdm(train_loader)):
            optimizer.zero_grad()
            outputs = model(bg).double()
            # print(label)
            
            label = torch.tensor([label]).to(device).double()
            
            # print(outputs.shape,label.shape)
            # print(outputs,label)
            # if iter == 2:
            #     break
            loss = loss_fn(outputs,label)
            # print("Loss",loss)
            loss.backward()
            
            optimizer.step()
            
        
            train_loss.append(loss.item())
        # for param in model.parameters():
        #     print(param.grad)
        
#         val_loss = []
#         model = model.eval()
#         with torch.no_grad():
#             for iter, (bg, label) in enumerate(tqdm(val_loader)):
#                 outputs = model(bg).double()
#                 label = torch.tensor(label).double()
#                 loss = loss_fn(outputs,label)
#                 val_loss.append(loss.item())
        
        train_loss = np.mean(train_loss)
        # val_loss = np.mean(val_loss)
    
        # print(f'Epoch {epoch}: train loss {train_loss} val loss {val_loss}')
        print(f'Epoch {epoch}: train loss {train_loss}')

train(model,train_dataset)


  0%|                                                   | 0/149 [00:00<?, ?it/s]/tmp/ipykernel_2638/3202670333.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.functional.softmax(x)
100%|████████████████████████████████████████| 149/149 [00:00<00:00, 227.19it/s]


Epoch 0: train loss 68.45637583892618


100%|████████████████████████████████████████| 149/149 [00:00<00:00, 223.80it/s]


Epoch 1: train loss 68.45637583892618


100%|████████████████████████████████████████| 149/149 [00:00<00:00, 225.32it/s]


...

Epoch 7: train loss 68.45637583892618


100%|████████████████████████████████████████| 149/149 [00:00<00:00, 222.21it/s]


Epoch 8: train loss 68.45637583892618


100%|████████████████████████████████████████| 149/149 [00:00<00:00, 233.12it/s]

Epoch 9: train loss 68.45637583892618
